In [1]:
import pandas as pd
import numpy as np

In [72]:
def my_func(A,y,gamma):
    zero_tol=1e-15
    n=A.shape[1]
    x=np.zeros(n)
    theta=x
    active_set=set()
    gram_matrix=np.dot(A.T,A)
    target_corr=np.dot(A.T,y)
    signs=np.zeros(n,dtype=np.int8)
    z_opt=np.inf
    nz_opt=0
    grad=-2*target_corr
    max_grad_zero=np.argmax(np.abs(grad))
    sds=np.dot(y.T,y)
    while z_opt>gamma or not np.allclose(nz_opt,0):
        if np.allclose(nz_opt,0):
            i=np.argmax(np.abs(grad)*(signs==0))
            if grad[i]>gamma:
                signs[i]=-1.
                x[i]=0
                active_set.add(i)
            elif grad[i]<-gamma:
                signs[i]=1.
                x[i]=0.
                active_set.add(i)
            if len(active_set)==0:
                break
        act_ind=np.array(sorted(active_set))
        A_h_t_A_h=gram_matrix[np.ix_(act_ind,act_ind)]
        A_h_t_y=target_corr[act_ind]
        theta_h=signs[act_ind]
        t1=np.linalg.inv(A_h_t_A_h)
        t2=A_h_t_y-(gamma/2)*theta_h
        x_new=np.dot(t1,t2)
        new_signs=np.sign(x_new)
        x_h=x[act_ind]
        sign_flips=np.where(abs(new_signs-theta_h)>1)[0]
        if len(sign_flips)>0:
            min_val=np.inf
            #min_val_curr=None
            min_val_curr=x_new
            min_val=(np.dot(y.T,y)+(np.dot(x_new,np.dot(x_new,np.dot(A_h_t_A_h,x_new))-2*np.dot(x_new,A_h__t_y))+gamma*abs(x_new).sum()))

        r_new_pt=0
        for ch in sign_flips:
            v1=x_new[ch]
            v2=x_h[ch]
            step=v2/(v2-v1)
            new_pt=x_h-step*(x_h-x_new)
            func_val=np.dot(y.T,y)+(np.dot(new_pt,np.dot(A_h_t_A_h,new_pt))-2*np.dot(new_pt,A_h_t_y)+gamma*abs(new_pt).sum())
            if func_val < min_val:
                min_val=func_val
                r_step=step
                r_new_pt=new_pt
        else:
            r_new_pt=x_new
        
        x[act_ind]=r_new_pt
        #print("print act_ind",act_ind)
        put_0=act_ind[np.abs(x[act_ind])<zero_tol]
        x[put_0]=0.
        signs[act_ind]=np.int8(np.sign(x[act_ind]))
        active_set.difference_update(put_0)
        grad=-2*target_corr+2*np.dot(gram_matrix,x)
        z_opt=np.max(abs(grad[signs==0]))
        nz_opt=np.max(abs(grad[signs!=0]+gamma*signs[signs!=0]))

    return x
    

In [78]:
A=np.array([[1,2,3],[4,5,6],[9,5,2]])

In [79]:
y=np.array([1,2,5])
gamma=0.2

In [80]:
my_func(A,y,gamma)

array([ 0.54756932,  0.        ,  0.00529348])